In [1]:
import os

In [ ]:
SCRIPT_DIR = os.path.dirname(os.path.abspath('/nlsasfs/home/gpucbh/vyakti11/iitrpr11/tasks'))

DATA_DIR = os.path.join(SCRIPT_DIR, 'data/Train')

DATA_DIR = os.path.normpath(DATA_DIR)

In [ ]:

import sys
import cv2
import numpy as np
import pandas as pd
from scipy.signal import detrend

def GREEN_ALGORITHM(signal, fps):
    """
    Green-Channel (G) rPPG Algorithm.
    
    Args:
        signal (np.array): Input RGB signal with shape (Frames, 3)
        fps (float): Frames per second
        
    Returns:
        np.array: The extracted rPPG waveform
    """
 
    green_channel = signal[:, 1]
    
    mean_g = np.mean(green_channel)
    if mean_g == 0:
        return green_channel
    
    h = (green_channel / mean_g) - 1

    h = detrend(h)
    
    return h



META_FILE = os.path.join(DATA_DIR, "labels_train.xlsx")

print(f" Script Location: {SCRIPT_DIR}")
print(f" Data Location:   {DATA_DIR}")

# Check if Data Directory exists
if not os.path.exists(DATA_DIR):
    print(f" Error: The directory '{DATA_DIR}' does not exist.")
    print("Please ensure your folder structure is correct.")
    sys.exit()

# ---- 3. FACE DETECTION ----
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def extract_face(frame):
    """Detects the largest face in the frame."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    x, y, w, h = max(faces, key=lambda b: b[2] * b[3])
    return frame[y:y+h, x:x+w]

def extract_rgb_signal(video_path):
    """Extracts the mean RGB signal from the face ROI."""
    cap = cv2.VideoCapture(video_path)
    rgb_signals = []

    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return np.array([])

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        face = extract_face(frame)
        
        if face is None:
            continue

        try:
            face = cv2.resize(face, (128, 128))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            mean_rgb = np.mean(face.reshape(-1, 3), axis=0)
            rgb_signals.append(mean_rgb)
        except Exception:
            continue

    cap.release()
    return np.array(rgb_signals)

def main():
    if not os.path.exists(META_FILE):
        print("Notice: Metadata file not found. Scanning 'data' folder for videos...")
        video_files = [f for f in os.listdir(DATA_DIR) if f.endswith(('.mp4', '.avi', '.mov'))]
        
        if not video_files:
            print(f"Error: No videos found in {DATA_DIR}.")
            return
            
        df = pd.DataFrame({'video': video_files, 'label': [0] * len(video_files)})
    else:
        df = pd.read_excel(META_FILE)

    results = {}

    for _, row in df.iterrows():
        video_name = row["video"]
        label = row["label"]
        video_path = os.path.join(DATA_DIR, video_name)

        print(f"Processing: {video_name}...")

        if not os.path.exists(video_path):
            print(f"Warning: File not found ({video_name}). Skipping.")
            continue

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()

        rgb_signal = extract_rgb_signal(video_path)

        if len(rgb_signal) < fps:
            print(f"Skipping {video_name}: Signal too short or face not detected.")
            continue

        try:
            rppg_waveform = GREEN_ALGORITHM(rgb_signal, fps)
            
            results[video_name] = {
                "label": label,
                "rppg": rppg_waveform
            }
            print(f"Success: {video_name} | Waveform Length: {len(rppg_waveform)}")

        except Exception as e:
            print(f"Error processing {video_name}: {e}")

    output_path = os.path.join(DATA_DIR, "rppg_waveforms_green.npy")
    if results:
        np.save(output_path, results)
        print(f"\nAll Green-channel signals have been saved to: {output_path}")
    else:
        print("\nNo data was saved.")

if __name__ == "__main__":
    main()

📂 Script Location: /nlsasfs/home/gpucbh/vyakti11/iitrpr11
📂 Data Location:   /nlsasfs/home/gpucbh/vyakti11/iitrpr11/data/Train
Processing: subject_10_Vid_6.avi...
Processing: subject_11_Vid_6.MP4...
Processing: subject_12_Vid_6.MP4...
Processing: subject_15_Vid_6.avi...
Processing: subject_16_Vid_6.MP4...
Processing: subject_18_Vid_6.webm...
Processing: subject_1_Vid_5.avi...
Processing: subject_20_Vid_2.avi...
Success: subject_20_Vid_2.avi | Waveform Length: 2719
Processing: subject_20_Vid_5_2.avi...
Processing: subject_20_Vid_6.avi...
Success: subject_20_Vid_6.avi | Waveform Length: 2407
Processing: subject_20_Vid_7.avi...
Processing: subject_21_Vid_5.avi...
Processing: subject_23_Vid_5.avi...
Success: subject_23_Vid_5.avi | Waveform Length: 5081
Processing: subject_3_Vid_2.avi...
Processing: subject_3_Vid_5.avi...
Processing: subject_3_Vid_7.avi...
Processing: subject_4_Vid_6.MP4...
Processing: subject_6_Vid_6.MP4...
Processing: subject_7_Vid_2.avi...
Processing: subject_7_Vid_4.avi